In [1]:
import keras

Adesso posso caricare il modello addestrato in precedenza e preoccuparmi semplicemente dell'output dell'ultima rete convoluzionale. Dal summary si può notare che l'ultima layer convoluzionale emette 500 valori relativi alle attivazioni, questi saranno quindi le feature del nostro nuovo modello.

In [2]:
model = keras.saving.load_model("../models/modello-passi-recurrent.keras")
attivazioni = keras.Model(inputs=model.layers[0].output,outputs=model.layers[-2].output)
attivazioni.summary()
window = model.layers[0].output.shape[1]
out = model.layers[-1].input.shape[1]

In [3]:
import pandas as pd
import scripts.config as cfg
import numpy as np

dati = pd.read_csv("../data/scalati.csv.gz")[:cfg.dataset_chunk].dropna()

Questa sotto è una visualizzazione delle attivazioni della rete in risposta all'input mostrato

In [4]:
import matplotlib.pyplot as plt

for uuid,group in dati.groupby('uuid')[["date","n_steps"]]:
    if len(group)<window:
        continue
    inp = group.values[-window:].reshape(window,2)
    pred= attivazioni.predict(np.array([inp]))
    plt.plot(inp[:,0], inp[:,1], label='Truth')
    plt.show()
    plt.close()
    plt.imshow(pred.reshape(window,-1).T,cmap='Greys')
    plt.show()
    plt.close()
    break

Preparo di nuovo i dati dei passi che dovrò dare in pasto alla rete per ottenere le attivazioni. Quelle attivazioni saranno poi i dati di training per il prossimo modello.

In [5]:
import numpy as np

xtrain = []
ytrain = []
xtest=[]
ytest=[]
test_ratio=0.1
for uuid, group in dati.groupby('uuid')[["date","n_steps","n_sleep_score"]]:
    split=int(len(group)*(1-test_ratio))
    if split<window:
        continue
        
    inputs = np.array(group[["date","n_steps"]].values)
    output = np.array(group["n_sleep_score"].values)


    for i in range (window,split):
        xtrain.append(inputs[i-window:i]) # Una finestra di <window> osservazioni sui passi
        ytrain.append(output[i]) # La qualità del sonno alla fine della finestra

    for i in range (split,len(group)):
        xtest.append(inputs[i-window:i])
        ytest.append(output[i])

xtrain=np.array(xtrain).reshape(-1,window,2)
ytrain=np.array(ytrain).reshape(-1,1)
xtest=np.array(xtest).reshape(-1,window,2)
ytest=np.array(ytest).reshape(-1,1)

Converto le osservazioni sui passi in una serie di attivazioni, da usare come input per il modello. Visto che 500 sono un po' tante ho provato anche a ridurre la dimensionalità con PCA

In [6]:
attivazioni_train = attivazioni.predict_on_batch(xtrain)
attivazioni_test = attivazioni.predict_on_batch(xtest)

In [7]:
# from sklearn.decomposition import PCA
# 
# pca= PCA(n_components=.95)
# attivazioni_train=pca.fit_transform(attivazioni_train)
# attivazioni_test= pca.transform(attivazioni_test)
# attivazioni_train

In [8]:
ytest=ytest.reshape(-1)
ytrain=ytrain.reshape(-1)
ytrain

Provo con un Percettrone multistrato

In [9]:
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge,LinearRegression,Lasso,SGDRegressor

regressore = Ridge()
regressore.fit(attivazioni_train,ytrain)

In [10]:
regressore.score(attivazioni_test,ytest)

In [11]:
dati = pd.read_csv("../data/scalati.csv.gz")[-10000:].dropna()


In [12]:
for uuid, group in dati.groupby('uuid')[["date","n_steps","n_sleep_score"]]:
    if len(group) < 2 * cfg.window:
        continue
    pred = []
    data = []
    for i in range(cfg.window, len(group)):
        data.append(group[["date","n_steps"]][i - cfg.window:i])
    data=np.array(data)
    pred = attivazioni.predict_on_batch(data)
    pred= regressore.predict(pred)
    plt.title("prediction for user: " + uuid)
    plt.plot(group.date.values, group.n_sleep_score.values, label='Truth')
    plt.plot(group.date[cfg.window:].values, pred, label='Predicted')
    plt.show()
    plt.close()